In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
dataset = pd.read_csv('Inputdata.csv')
dataset.head()
X = dataset.iloc[:,[3,4,5,6,7,8,9,10,11,12,13,14]].values
y = dataset.iloc[:,[2]].values 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)  # 70% train, 30% remaining
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)  # Split remaining 30% equally

# Apply Standard Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
from ngboost import NGBoost
from ngboost.distns import Normal   # you are using Normal
from sklearn.metrics import mean_squared_error, brier_score_loss
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import calibration_curve
from scipy.optimize import differential_evolution
import numpy as np
import matplotlib.pyplot as plt

# --------------------------
# 1. DE optimisation (your code, just slightly tidied)
# --------------------------
def fitness_function(params, X_train, y_train, X_val, y_val):
    n_estimators = int(params[0])
    learning_rate = params[1]

    model = NGBoost(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        Dist=Normal   # explicit, same as your original import
    )
    model.fit(X_train, y_train)

    preds_val = model.predict(X_val)
    mse = mean_squared_error(y_val, preds_val)
    return mse  # DE minimises this

lb = [50, 0.01]
ub = [200, 0.2]

result = differential_evolution(
    fitness_function,
    bounds=list(zip(lb, ub)),
    args=(X_train, y_train, X_val, y_val),
    strategy='best1bin',
    popsize=40,
    maxiter=100,
    mutation=(0.5, 1),
    recombination=0.7,
    seed=42
)

best_solution = result.x
best_fitness = result.fun

print(f"Optimized hyperparameters: n_estimators={int(best_solution[0])}, "
      f"learning_rate={best_solution[1]:.4f}")
print(f"Minimum validation MSE: {best_fitness:.4f}")

# Train best NGBoost on full training data
best_regressor = NGBoost(
    n_estimators=int(best_solution[0]),
    learning_rate=best_solution[1],
    Dist=Normal
)
best_regressor.fit(X_train, y_train)

pred_test_uncal = best_regressor.predict(X_test)
test_mse = mean_squared_error(y_test, pred_test_uncal)
print(f"Test MSE with best parameters: {test_mse:.4f}")


# --------------------------
# 2. Convert regression output to probabilities
#    (for binary 0/1 target)
# --------------------------
# Predictions on val and test
pred_val_uncal = best_regressor.predict(X_val)
pred_test_uncal = best_regressor.predict(X_test)

# Interpret as "scores" and clip to [0,1]
proba_val_uncal = np.clip(pred_val_uncal, 0.0, 1.0)
proba_test_uncal = np.clip(pred_test_uncal, 0.0, 1.0)


# --------------------------
# 3. Isotonic calibration on validation set
# --------------------------
iso = IsotonicRegression(out_of_bounds='clip')
iso.fit(proba_val_uncal, y_val)

# Calibrated probabilities on test set
proba_test_cal = iso.predict(proba_test_uncal)


# --------------------------
# 4. Reliability curves (before vs after, with quantile bins)
# --------------------------
prob_true_uncal, prob_pred_uncal = calibration_curve(
    y_test, proba_test_uncal, n_bins=15, strategy='quantile'
)

prob_true_cal, prob_pred_cal = calibration_curve(
    y_test, proba_test_cal, n_bins=15, strategy='quantile'
)

plt.figure(figsize=(6, 6))
plt.plot(prob_pred_uncal, prob_true_uncal, 'o-', label='Uncalibrated NGBoost (DE)')
plt.plot(prob_pred_cal, prob_true_cal, 's-', label='Calibrated NGBoost (Isotonic)')
plt.plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel("Predicted probability")
plt.ylabel("Observed frequency")
plt.title("Reliability diagram - NGBoost (DE-tuned)")
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()


# --------------------------
# 5. Calibration metrics: Brier, ECE, sharpness
# --------------------------
def expected_calibration_error(y_true, y_prob, n_bins=15):
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    bin_ids = np.digitize(y_prob, bins) - 1
    ece = 0.0
    n = len(y_true)
    for b in range(n_bins):
        mask = bin_ids == b
        if np.any(mask):
            prob_mean = y_prob[mask].mean()
            freq_mean = y_true[mask].mean()
            ece += (mask.sum() / n) * abs(prob_mean - freq_mean)
    return ece

brier_uncal = brier_score_loss(y_test, proba_test_uncal)
brier_cal   = brier_score_loss(y_test, proba_test_cal)

ece_uncal = expected_calibration_error(y_test, proba_test_uncal)
ece_cal   = expected_calibration_error(y_test, proba_test_cal)

sharp_uncal = np.var(proba_test_uncal)
sharp_cal   = np.var(proba_test_cal)

print("=== Calibration metrics (NGBoost-DE) ===")
print(f"Uncalibrated - Brier: {brier_uncal:.4f}, ECE: {ece_uncal:.4f}, Sharpness: {sharp_uncal:.4f}")
print(f"Calibrated   - Brier: {brier_cal:.4f}, ECE: {ece_cal:.4f}, Sharpness: {sharp_cal:.4f}")